# Semantic Textual Similarity


# Problem Statement 



The data contains a pair of paragraphs. These text paragraphs are randomly sampled from a raw dataset. Each pair of       sentences may or may not be semantically similar. The candidate is to predict a value between 0-1 indicating the similarity between the pair of text. A sample of a similar dataseet will be used as test data, therefore it's crucial to the model solution using provided dataset. Build an algorithm/model that can quantify the degree of similarity between the two text-based on Semantic similarity. Semantic Textual Similarity (STS) assesses the degree to which two sentences are semantically equivalent to each other. 1 means highly similar 0 means highly dissimilar.



# Import some library

In [4]:
import numpy as np
import pandas as pd
import collections
import re
from tqdm import tqdm
from sklearn.cluster import KMeans

from nltk.stem import WordNetLemmatizer  # For Lemmetization of words
from nltk.corpus import stopwords  # Load the list of stopwords
from nltk import word_tokenize # Convert paragraph in tokens

from gensim.models import word2vec # represent words in vectors
import gensim

# Read Data-Set

In [5]:
# Reading the given data-set using pandas
text_data = pd.read_csv("Precily_Text_Similarity.csv")
print("shape of text_data : ", text_data.shape)
text_data.head()

shape of text_data :  (3000, 3)


,Unique_ID,text1,text2
0,0,broadband challenges tv viewing the number of ...,gardener wins double in glasgow britain s jaso...
1,1,rap boss arrested over drug find rap mogul mar...,amnesty chief laments war failure the lack of ...
2,2,player burn-out worries robinson england coach...,hanks greeted at wintry premiere hollywood sta...
3,3,hearts of oak 3-2 cotonsport hearts of oak set...,redford s vision of sundance despite sporting ...
4,4,sir paul rocks super bowl crowds sir paul mcca...,mauresmo opens with victory in la amelie maure...


In [6]:
text_data.isnull().sum() # Check if text data have any null values

Unique_ID    0
text1        0
text2        0
dtype: int64

# Preprocessing of text1 & text2


* Removing punctuations like .,!$()*%@

* Removing stop words

* Lower casing

* Tokenization

* Stemming

* Lemmatization


In [7]:
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [8]:
# Combining all the above statements

preprocessed_text1 = []

# tqdm is for printing the status bar

for sentance in tqdm(text_data['text1'].values):
    sent = decontracted(sentance)
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)

    sent = ' '.join(e for e in sent.split() if e not in stopwords.words('english'))
    preprocessed_text1.append(sent.lower().strip())

100%|█████████████████████████████████████████| 3000/3000 [05:30<00:00,  9.08it/s]


In [9]:
# merging preprocessed_text1 in text_data

text_data['text1'] = preprocessed_text1
text_data.head()

,Unique_ID,text1,text2
0,0,broadband challenges tv viewing number europea...,gardener wins double in glasgow britain s jaso...
1,1,rap boss arrested drug find rap mogul marion s...,amnesty chief laments war failure the lack of ...
2,2,player burn worries robinson england coach and...,hanks greeted at wintry premiere hollywood sta...
3,3,hearts oak 3 2 cotonsport hearts oak set ghana...,redford s vision of sundance despite sporting ...
4,4,sir paul rocks super bowl crowds sir paul mcca...,mauresmo opens with victory in la amelie maure...


In [10]:
# combining all the above stundents 
preprocessed_text2 = []

# tqdm is for printing the status bar

for sentance in tqdm(text_data['text2'].values):
    sent = decontracted(sentance)
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
   
    sent = ' '.join(e for e in sent.split() if e not in stopwords.words('english'))
    preprocessed_text2.append(sent.lower().strip())

100%|█████████████████████████████████████████| 3000/3000 [05:20<00:00,  9.35it/s]


In [11]:
# merging preprocessed_text2 in text_data

text_data['text2'] = preprocessed_text2
text_data.head()

,Unique_ID,text1,text2
0,0,broadband challenges tv viewing number europea...,gardener wins double glasgow britain jason gar...
1,1,rap boss arrested drug find rap mogul marion s...,amnesty chief laments war failure lack public ...
2,2,player burn worries robinson england coach and...,hanks greeted wintry premiere hollywood star t...
3,3,hearts oak 3 2 cotonsport hearts oak set ghana...,redford vision sundance despite sporting cordu...
4,4,sir paul rocks super bowl crowds sir paul mcca...,mauresmo opens victory la amelie mauresmo mari...


In [12]:
def word_tokenizer(text):
            tokens = word_tokenize(text) #tokenize the text
            lemmatizer = WordNetLemmatizer() #stems the text
            tokens = [lemmatizer.lemmatize(t) for t in tokens]
            return tokens

# Proposed Approach



# Word Embedding Algorithms


* Word embeddings are in fact a class of techniques where individual words are represented as real-valued vectors in a predefined vector space. Pre-trained word embeddings are also available in the word2vec code.google page.

*  In this I am using Google news pre trained vectors and compare similarity between text1 & text2 using n_similarity method in gensim library which is nothing compares cosine similarity between two.

In [14]:
# Loading pre_trained Google News Vectors 

wordmodelfile="GoogleNews-vectors-negative300.bin.gz"
wordmodel= gensim.models.KeyedVectors.load_word2vec_format(wordmodelfile, binary=True)

In [15]:
# This code check if word in text1 & text2 present in our google news vectors vocabalry.
# if not it removes that word and if present it compares similarity score between text1 and text2 words


similarity = [] # List for store similarity score

for ind in text_data.index:
    
        s1 = text_data['text1'][ind]
        s2 = text_data['text2'][ind]
        
        if s1==s2:
                 similarity.append(0.0) # 0 means highly similar
                
        else:   

            s1words = word_tokenizer(s1)
            s2words = word_tokenizer(s2)
            
           
            vocab = wordmodel.key_to_index  #the vocabulary considered in the word embeddings
            
            if len(s1words and s2words)==0:
                    similarity.append(1.0)

            else:
                
                for word in s1words.copy(): #remove sentence words not found in the vocab
                    if (word not in vocab):
                           
                            
                            s1words.remove(word)
                        
                    
                for word in s2words.copy(): #idem

                    if (word not in vocab):
                           
                            s2words.remove(word)
                            
                            
                similarity.append((1-wordmodel.n_similarity(s1words, s2words))) # as it is given 1 means highly dissimilar & 0 means highly similar
       



# Final Submission




• Make Final DataFrame and save a CSV file of similarity scores with Unique_ID.(Columns : Unique_ID, Similarity_Score)

In [16]:
# Get Unique_ID and similarity

final_score = pd.DataFrame({'Unique_ID':text_data.Unique_ID, 'Similarity_score':similarity})
final_score.head() 

,Unique_ID,Similarity_score
0,0,0.314564
1,1,0.366671
2,2,0.282510
3,3,0.383152
4,4,0.207984


In [17]:
# save df as CSV file

final_score.to_csv('final_score.csv',index=False)